In [1]:
# Basic Python package
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string 
import re

In [2]:
# File paths
train_file = r"C:\Users\prits\Downloads\Data\cleaned_ghc_train.csv"
test_file = r'C:\Users\prits\Downloads\Data\cleaned_ghc_test.csv'

# Load CSV files into pandas DataFrames
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

In [3]:
print(train_df)

                                                    text  hd  cv  vo label
0      he most likely converted to islam due to his n...   0   0   0     h
1      so ford lied about being a psychologist record...   0   0   0     h
2      jobs education ending abuse of nation californ...   0   0   0    nh
3      i share a lot of your values  like many who do...   0   0   0     h
4      i am so ready to get back to blogging  recipes...   0   0   0    nh
...                                                  ...  ..  ..  ..   ...
21771  im a fan of western civilization and one bedro...   0   0   0     h
21772  or  is she saying that muslims dont know how t...   0   0   0     h
21773  thank you to all my followers that follow me e...   0   0   0     h
21774                                   wednesday music    0   0   0    nh
21775                    this is a really big surprise     0   0   0     h

[21776 rows x 5 columns]


In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [6]:
#word tokenisation
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

train_df['text'] = train_df['text'].astype(str)
# Download the necessary NLTK data files (only need to do this once)
nltk.download('punkt')

# Tokenize each text in the DataFrame
train_df['tokens'] = train_df['text'].apply(word_tokenize)

# Print the DataFrame with tokens
print(train_df)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prits\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                                    text  hd  cv  vo label  \
0      he most likely converted to islam due to his n...   0   0   0     h   
1      so ford lied about being a psychologist record...   0   0   0     h   
2      jobs education ending abuse of nation californ...   0   0   0    nh   
3      i share a lot of your values  like many who do...   0   0   0     h   
4      i am so ready to get back to blogging  recipes...   0   0   0    nh   
...                                                  ...  ..  ..  ..   ...   
21771  im a fan of western civilization and one bedro...   0   0   0     h   
21772  or  is she saying that muslims dont know how t...   0   0   0     h   
21773  thank you to all my followers that follow me e...   0   0   0     h   
21774                                   wednesday music    0   0   0    nh   
21775                    this is a really big surprise     0   0   0     h   

                                                  tokens  
0   

 TF-IDF stands for Term Frequency Inverse Document Frequency of records. It can be defined as the calculation of how relevant a word in a series or corpus is to a text. The meaning increases proportionally to the number of times in the text a word appears but is compensated by the word frequency in the corpus (data-set).

In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


texts = train_df['text'].values
labels = train_df['label'].values

# Create a mask for rows without NaN values in 'texts'
mask = ~pd.isnull(texts)

# Apply the mask to both texts and labels
texts = texts[mask]
labels = labels[mask]

def tfidf_embedding(texts):
    vectorizer = TfidfVectorizer(max_features=5000)
    embeddings = vectorizer.fit_transform(texts)
    return embeddings

embeddings_tfidf = tfidf_embedding(texts)
print(embeddings_tfidf)


  (0, 4785)	0.16203292794897928
  (0, 2901)	0.3892354001637881
  (0, 1335)	0.3892354001637881
  (0, 2337)	0.31046282407931214
  (0, 1774)	0.12408419299574001
  (0, 498)	0.21140455796475746
  (0, 2946)	0.3260066957099285
  (0, 2118)	0.18885372906548606
  (0, 1403)	0.29776487683266445
  (0, 2336)	0.27002468653824596
  (0, 4498)	0.18298530207380456
  (0, 2601)	0.3060515584997593
  (0, 2875)	0.22706220952863765
  (0, 2056)	0.1789671016915568
  (1, 4902)	0.2613628716585361
  (1, 2990)	0.19093982502863016
  (1, 4243)	0.35324032765395047
  (1, 2417)	0.18222690689730897
  (1, 3955)	0.23702603418299092
  (1, 3890)	0.3286745616687188
  (1, 3592)	0.37005381211840016
  (1, 89)	0.18455570647653158
  (1, 2593)	0.3834673281604326
  (1, 1779)	0.35822471249323373
  (1, 4073)	0.18314506744675355
  :	:
  (21773, 3122)	0.2300645094202304
  (21773, 2915)	0.14194880498801446
  (21773, 2348)	0.0748393265299162
  (21773, 466)	0.25211483374965005
  (21773, 2955)	0.12316533172556458
  (21773, 689)	0.09300220080

Word2Vec is an effort to map words to high-dimensional vectors to capture the semantic relationships between words.Words with similar meanings should have similar vector representations, according to the main principle of Word2Vec.

In [8]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


# Extract texts from the training dataframe
texts_train = train_df['text'].values

# Handle any NaN values in the text column
texts_train = np.where(pd.isnull(texts_train), '', texts_train)

# Tokenize the texts
tokenized_train = [word_tokenize(text) for text in texts_train]

# Train a Word2Vec model on the training texts
w2v_model = Word2Vec(sentences=tokenized_train, vector_size=100, window=5, min_count=1, workers=4)

# Function to create average Word2Vec embeddings for each text
def get_average_word2vec(tokens_list, model, vocabulary, num_features):
    feature_vec = np.zeros((num_features,), dtype="float32")
    n_words = 0
    for token in tokens_list:
        if token in vocabulary:
            n_words += 1
            feature_vec = np.add(feature_vec, model.wv[token])
    if n_words > 0:
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

# Create embeddings for the training texts
vocabulary = set(w2v_model.wv.index_to_key)
embeddings_train = np.array([get_average_word2vec(tokens, w2v_model, vocabulary, 100) for tokens in tokenized_train])

print(embeddings_train)


[[ 0.12863833  0.18229553  0.03983613 ... -0.32184178 -0.06998452
  -0.29096565]
 [ 0.01588773  0.26588994  0.19004579 ... -0.25425884 -0.08934192
  -0.5033654 ]
 [-0.04830959  0.24815992 -0.02696376 ... -0.10394708  0.07483985
  -0.07230787]
 ...
 [-0.09826891  0.53646976  0.35280833 ... -0.28204152 -0.09486767
  -0.17183422]
 [-0.00709109  0.30483574 -0.04895918 ... -0.27518755  0.08552052
  -0.19927996]
 [-0.10682079  0.2881086   0.05734314 ... -0.43156347 -0.1361157
  -0.6704223 ]]


OneHotEncoding:One hot encoding is a technique that we use to represent categorical variables as numerical values 

In [9]:

import pandas as pd
from sklearn.preprocessing import OneHotEncoder


print(train_df.head(2), sep = " ")
train_df.info()

# Converting 'IsHatespeech' column to categorical to avoid Bias 
train_df['text'] = train_df['label'].astype(str)

encoder = OneHotEncoder()

encoded_data = encoder.fit_transform(train_df[['label']])

# Converting the encoded data into an array
encoded_array = encoded_data.toarray()

# Print the encoded data (this will show the transformed categorical data)
encoded_array

                                                text  hd  cv  vo label  \
0  he most likely converted to islam due to his n...   0   0   0     h   
1  so ford lied about being a psychologist record...   0   0   0     h   

                                              tokens  
0  [he, most, likely, converted, to, islam, due, ...  
1  [so, ford, lied, about, being, a, psychologist...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21776 entries, 0 to 21775
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    21776 non-null  object
 1   hd      21776 non-null  int64 
 2   cv      21776 non-null  int64 
 3   vo      21776 non-null  int64 
 4   label   21776 non-null  object
 5   tokens  21776 non-null  object
dtypes: int64(3), object(3)
memory usage: 1020.9+ KB


array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

Label Encoding :
Label Encoding is a technique that is used to convert categorical columns into numerical ones so that they can be fitted by machine learning models which only take numerical data

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit the encoder on the categorical features
encoded_data = label_encoder.fit_transform(train_df[['label']])

print(encoded_data)

[0 0 1 ... 0 1 0]


C:\Users\prits\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
